Tutorial - Text classification with tensorflow hub : movie reviews

In [1]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print('Version: ', tf.__version__)
print('Eager mode: ', tf.executing_eagerly())
print('Hub version: ', hub.__version__)
print('GPU is', 'available' if tf.config.experimental.list_physical_devices('GPU') else 'NOT Available')

Version:  2.4.0
Eager mode:  True
Hub version:  0.11.0
GPU is NOT Available


Download IMDB dataset
: contains the text of 50000 movie reviews from the internet Movie database

In [2]:
# split the training set into 60% and 40% so, we will end up with 
# 15000 examples for training, 10000 examples for validation, and 25000 examples for testing

train_data, validation_data, test_data = tfds.load(
    name = 'imdb_reviews',
    split = ('train[:60%]', 'train[60%:]', 'test'),
    as_supervised = True
)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteOWOTPQ/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteOWOTPQ/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteOWOTPQ/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


Explore the data

In [3]:
# print the first 10 examples
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [4]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

Build A model

1. How to represent the text
2. How many layers to use in the model
3. How many Hidden units to use for each layer

Use pre trained text embedding model from TF Hub

In [5]:
embedding = 'https://tfhub.dev/google/nnlm-en-dim50/2'
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [6]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


Loss function and optimizer

In [8]:
model.compile(optimizer= 'adam', 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy']
              )

Train the model
Train the model for 10 epochs in mini-batches of 512 samples.

In [10]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 19s 577ms/step - loss: 0.7117 - accuracy: 0.5018 - val_loss: 0.6548 - val_accuracy: 0.5028
Epoch 2/10
30/30 [==============================] - 17s 559ms/step - loss: 0.6213 - accuracy: 0.5305 - val_loss: 0.5414 - val_accuracy: 0.6899
Epoch 3/10
30/30 [==============================] - 17s 560ms/step - loss: 0.4791 - accuracy: 0.7575 - val_loss: 0.4274 - val_accuracy: 0.7830
Epoch 4/10
30/30 [==============================] - 17s 559ms/step - loss: 0.3418 - accuracy: 0.8538 - val_loss: 0.3572 - val_accuracy: 0.8362
Epoch 5/10
30/30 [==============================] - 17s 558ms/step - loss: 0.2428 - accuracy: 0.9136 - val_loss: 0.3249 - val_accuracy: 0.8585
Epoch 6/10
30/30 [==============================] - 17s 567ms/step - loss: 0.1759 - accuracy: 0.9440 - val_loss: 0.3115 - val_accuracy: 0.8628
Epoch 7/10
30/30 [==============================] - 17s 560ms/step - loss: 0.1306 - accuracy: 0.9620 - val_loss: 0.3099 - val_accuracy: 0.8631

Evaluate the model

In [11]:
results = model.evaluate(test_data.batch(512), verbose=2)

results

49/49 - 3s - loss: 0.3557 - accuracy: 0.8532


[0.35574108362197876, 0.8532000184059143]

In [12]:
for name, value in zip(model.metrics_names, results):
  print('{}: {:.3f}'.format(name, value))

loss: 0.356
accuracy: 0.853
